# Pandas json_normalize()

This is a notebook for the medium article [Pandas json_normalize()](https://bindichen.medium.com/all-pandas-json-normalize-you-should-know-for-flattening-json-13eae1dfb7dd)

Please check out article for instructions

**License**: [BSD 2-Clause](https://opensource.org/licenses/BSD-2-Clause)

In [1]:
import pandas as pd

## 1. Flattening a simple JSON

### A dict

In [2]:
a_dict = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
}

In [3]:
df = pd.json_normalize(a_dict)
df

,school,location,ranking
0,ABC primary school,London,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   school    1 non-null      object
 1   location  1 non-null      object
 2   ranking   1 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 152.0+ bytes


### A list of dicts

In [5]:
json_list = [
    { 'class': 'Year 1', 'num_of_students': 20, 'room': 'Yellow' },
    { 'class': 'Year 2', 'num_of_students': 25, 'room': 'Blue' },
]
pd.json_normalize(json_list)

,class,num_of_students,room
0,Year 1,20,Yellow
1,Year 2,25,Blue


In [6]:
# Missing field is represented as NaN
json_list = [
    { 'class': 'Year 1', 'num_of_students': 20, 'room': 'Yellow' },
    { 'class': 'Year 2', 'room': 'Blue' },
]
pd.json_normalize(json_list)

,class,num_of_students,room
0,Year 1,20.0,Yellow
1,Year 2,NaN,Blue


## 2. Flattening a JSON with Multiple levels

### A dict

In [7]:
a_obj = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
    'info': {
        'president': 'John Kasich',
        'contacts': {
          'email': {
              'admission': 'admission@abc.com',
              'general': 'info@abc.com'
          },
          'tel': '123456789',
      }
    }
}

In [8]:
pd.json_normalize(a_obj)

,school,location,ranking,info.president,info.contacts.email.admission,info.contacts.email.general,info.contacts.tel
0,ABC primary school,London,2,John Kasich,admission@abc.com,info@abc.com,123456789


In [9]:
# If you don’t want to dig all the way down into each sub-object use the max_level argument.
pd.json_normalize(a_obj, max_level=1)

,school,location,ranking,info.president,info.contacts
0,ABC primary school,London,2,John Kasich,"{'email': {'admission': 'admission@abc.com', '..."


### A list of dicts

In [10]:
json_list = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask' 
            }
        }
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'math': 'Alan Turing', 
                'physics': 'Albert Einstein' 
            }
        }
    },
]

pd.json_normalize(json_list)

,class,student count,room,info.teachers.math,info.teachers.physics
0,Year 1,20,Yellow,Rick Scott,Elon Mask
1,Year 2,25,Blue,Alan Turing,Albert Einstein


In [11]:
pd.json_normalize(json_list, max_level=1)

,class,student count,room,info.teachers
0,Year 1,20,Yellow,"{'math': 'Rick Scott', 'physics': 'Elon Mask'}"
1,Year 2,25,Blue,"{'math': 'Alan Turing', 'physics': 'Albert Ein..."


In [12]:
# with missing keys
json_list = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 'math': 'Rick Scott', 'physics': 'Elon Mask' }
        }
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 'physics': 'Albert Einstein' }
        }
    },
]

pd.json_normalize(json_list)

,class,student count,room,info.teachers.math,info.teachers.physics
0,Year 1,20,Yellow,Rick Scott,Elon Mask
1,Year 2,25,Blue,NaN,Albert Einstein


## 3. Flattening a JSON with nested list

### A dict

In [13]:
a_dict = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
    'info': {
        'president': 'John Kasich',
        'contacts': {
          'email': {
              'admission': 'admission@abc.com',
              'general': 'info@abc.com'
          },
          'tel': '123456789',
      }
    },
    'students': [
      { 'name': 'Tom' },
      { 'name': 'James' },
      { 'name': 'Jacqueline' }
    ],
}

In [14]:
pd.json_normalize(a_dict)

,school,location,ranking,students,info.president,info.contacts.email.admission,info.contacts.email.general,info.contacts.tel
0,ABC primary school,London,2,"[{'name': 'Tom'}, {'name': 'James'}, {'name': ...",John Kasich,admission@abc.com,info@abc.com,123456789


In [15]:
# Flatten students
pd.json_normalize(a_dict, record_path =['students'])

,name
0,Tom
1,James
2,Jacqueline


In [16]:
# Include meta data
pd.json_normalize(
    a_dict, 
    record_path =['students'],
    meta=['school', ['info', 'contacts', 'tel']],
)

,name,school,info.contacts.tel
0,Tom,ABC primary school,123456789
1,James,ABC primary school,123456789
2,Jacqueline,ABC primary school,123456789


### A list of dicts

In [17]:
json_list = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask' 
            }
        },
        'students': [
            { 
                'name': 'Tom', 
                'sex': 'M', 
                'grades': { 'math': 66, 'physics': 77 } 
            },
            { 
                'name': 'James', 
                'sex': 'M', 
                'grades': { 'math': 80, 'physics': 78 } 
            },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'math': 'Alan Turing', 
                'physics': 'Albert Einstein' 
            }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

pd.json_normalize(json_list)

,class,student count,room,students,info.teachers.math,info.teachers.physics
0,Year 1,20,Yellow,"[{'name': 'Tom', 'sex': 'M', 'grades': {'math'...",Rick Scott,Elon Mask
1,Year 2,25,Blue,"[{'name': 'Tony', 'sex': 'M'}, {'name': 'Jacqu...",Alan Turing,Albert Einstein


In [18]:
pd.json_normalize(json_list, record_path =['students'])

,name,sex,grades.math,grades.physics
0,Tom,M,66.0,77.0
1,James,M,80.0,78.0
2,Tony,M,NaN,NaN
3,Jacqueline,F,NaN,NaN


In [19]:
pd.json_normalize(
    json_list, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']]
)

,name,sex,grades.math,grades.physics,class,room,info.teachers.math
0,Tom,M,66.0,77.0,Year 1,Yellow,Rick Scott
1,James,M,80.0,78.0,Year 1,Yellow,Rick Scott
2,Tony,M,NaN,NaN,Year 2,Blue,Alan Turing
3,Jacqueline,F,NaN,NaN,Year 2,Blue,Alan Turing


## 4. Ignoring KeyError if keys are not always present 

In [20]:
data = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask',
            }
        },
        'students': [
            { 'name': 'Tom', 'sex': 'M' },
            { 'name': 'James', 'sex': 'M' },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'physics': 'Albert Einstein'
            }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

In [21]:
pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']],
)

KeyError: "Try running with errors='ignore' as key 'math' is not always present"

In [22]:
pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']],
    errors='ignore'
)

,name,sex,class,room,info.teachers.math
0,Tom,M,Year 1,Yellow,Rick Scott
1,James,M,Year 1,Yellow,Rick Scott
2,Tony,M,Year 2,Blue,NaN
3,Jacqueline,F,Year 2,Blue,NaN


### 5. Custom separator using `sep`

In [23]:
data = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 'math': 'Rick Scott', 'physics': 'Elon Mask' }
        },
        'students': [
            { 'name': 'Tom', 'sex': 'M', 'grades': { 'math': 66, 'physics': 77 } },
            { 'name': 'James', 'sex': 'M', 'grades': { 'math': 80, 'physics': 78 } },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 'math': 'Alan Turing', 'physics': 'Albert Einstein' }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

In [24]:
pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']],
    sep='->'
)

,name,sex,grades->math,grades->physics,class,room,info->teachers->math
0,Tom,M,66.0,77.0,Year 1,Yellow,Rick Scott
1,James,M,80.0,78.0,Year 1,Yellow,Rick Scott
2,Tony,M,NaN,NaN,Year 2,Blue,Alan Turing
3,Jacqueline,F,NaN,NaN,Year 2,Blue,Alan Turing


## 6. Adding prefix for meta and record data using `meta_prefix` and `record_prefix`

In [25]:
pd.json_normalize(
    data, 
    record_path=['students'], 
    meta=['class'],
    meta_prefix='meta-',
    record_prefix='student-'
)

,student-name,student-sex,student-grades.math,student-grades.physics,meta-class
0,Tom,M,66.0,77.0,Year 1
1,James,M,80.0,78.0,Year 1
2,Tony,M,NaN,NaN,Year 2
3,Jacqueline,F,NaN,NaN,Year 2


## 7. Working with a local file

In [27]:
import json
# load data using Python JSON module
with open('data/simple.json','r') as f:
    data = json.loads(f.read())
    
# Normalizing data
pd.json_normalize(data)

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78


## 8. Working with a URL

In [28]:
import requests

URL = 'http://raw.githubusercontent.com/BindiChen/machine-learning/master/data-analysis/027-pandas-convert-json/data/simple.json'

data = json.loads(requests.get(URL).text)
pd.json_normalize(data) 

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78
